In [4]:
import ee
import geopandas as gpd
import geemap
import xarray as xr
import wxee

In [5]:
ee.Authenticate()
ee.Initialize(project='ee-nunezrimedio-tesina',opt_url='https://earthengine-highvolume.googleapis.com')

In [6]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

def get_province_geometry(province_name):
    provinces = ee.FeatureCollection("FAO/GAUL/2015/level1")
    return provinces.filter(ee.Filter.And(
        ee.Filter.eq('ADM0_NAME', 'Argentina'),
        ee.Filter.eq('ADM1_NAME', province_name)
    )).first().geometry()

# Get geometries for Santiago del Estero and Chaco
santiago_geometry = get_province_geometry('Santiago Del Estero')
chaco_geometry = get_province_geometry('Chaco')

# Find an area on the border
#border_area = santiago_geometry.intersection(chaco_geometry).centroid().buffer(3000).bounds()

border_area = ee.Geometry.Polygon([
    [
        [-61.831221, -25.658720],    # Northwest corner
        [-61.831221, -25.778083],    # Southwest corner
        [-61.711858, -25.778083],    # Southeast corner
        [-61.711858, -25.658720]     # Northeast corner
    ]
])


parana_delta = ee.Geometry.Polygon(
    [[[-59.5, -33.5],
      [-59.5, -34.0],
      [-58.8, -34.0],
      [-58.8, -33.5]]])

# Create a map centered on Argentina
Map = geemap.Map(center=[-27, -62], zoom=6)

# Add layers to the map
Map.addLayer(border_area, {'color': 'FF0000', 'fillColor': 'FF000088'}, 'Border Area')
Map.addLayer(parana_delta, {'color': '0000FF', 'fillColor': '0000FF88'}, 'Paraná Delta')

# Display the map
Map

Map(center=[-27, -62], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [7]:
# Define your area of interest
aoi = parana_delta

# Get ESA WorldCover image and clip it to your area
worldcover = ee.ImageCollection("ESA/WorldCover/v100").first()  # Get the single image
worldcover_clipped = worldcover.clip(aoi)

# Add the wx accessor to the image and convert to xarray
ds = worldcover_clipped.wx.to_xarray(region=ee.Feature(aoi).geometry(), scale=30)

# Apply the same data processing
ds = ds.fillna(-1)
ds = ds.clip(min=-128, max=127)
ds["Map"] = ds["Map"].astype("int8")  # Note: using "Map" instead of "label" as that's the band name in WorldCover
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")

# Save to netCDF file
ds.to_netcdf(r"Data\ESA\parana_delta_worldcover.nc")

Downloading:   0%|          | 0.00/462k [00:00<?, ?iB/s]

In [8]:
# Define your area of interest
aoi = border_area

# Get ESA WorldCover image and clip it to your area
worldcover = ee.ImageCollection("ESA/WorldCover/v100").first()  # Get the single image
worldcover_clipped = worldcover.clip(aoi)

# Add the wx accessor to the image and convert to xarray
ds = worldcover_clipped.wx.to_xarray(region=ee.Feature(aoi).geometry(), scale=30)

# Apply the same data processing
ds = ds.fillna(-1)
ds = ds.clip(min=-128, max=127)
ds["Map"] = ds["Map"].astype("int8")  # Note: using "Map" instead of "label" as that's the band name in WorldCover
ds["x"] = ds["x"].astype("float32")
ds["y"] = ds["y"].astype("float32")

# Save to netCDF file
ds.to_netcdf(r"Data\ESA\border_area_worldcover.nc")

Downloading:   0%|          | 0.00/3.41k [00:00<?, ?iB/s]